In [1]:
import gc
import re
import time
import warnings
from contextlib import contextmanager

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymysql
import seaborn as sns
from feature_engine import categorical_encoders as ce
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from datetime import datetime
import uuid
from pandas import DataFrame
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.simplefilter(action='ignore')


/Users/mvahit/anaconda3/envs/home_credit/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [3]:
# Display plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance",
                                                                                                   ascending=False)[
           :100].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    print(best_features)
    plt.figure(figsize=(15, 20))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')


def load_dataset(file_path, index=0):
    df = pd.read_csv(file_path, index_col=index)
    return df


def get_categoric_columns(df):
    cols = df.select_dtypes(include=['object', 'category']).columns
    return cols


def apply_label_encoding(l_df, columns):
    lbe = LabelEncoder()

    for col in columns:
        l_df[col] = lbe.fit_transform(l_df[col])

    return l_df


def apply_one_hot_encoding(l_df):
    original_columns = list(l_df)  # col names as string in a list
    categorical_columns = get_categoric_columns(l_df)  # categorical col names
    l_df = pd.get_dummies(l_df, columns=categorical_columns, drop_first=True)  # creating dummies
    new_columns = [c for c in l_df.columns if c not in original_columns]  # new col names
    return l_df, new_columns


# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


def rare_encoding(data, variables, rare_threshold=0.05, n_rare_categories=4):
    encoder = ce.RareLabelCategoricalEncoder(tol=rare_threshold, n_categories=n_rare_categories, variables=variables,
                                             replace_with='Rare')
    # fit the encoder
    encoder.fit(data)

    # transform the data
    data = encoder.transform(data)

    return data


def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
            to reduce memory usage.
        """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    # code takenn from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
    return df


def feature_early_shutdown(row):
    early_shutdown = 0
    if row.CREDIT_ACTIVE == "Active" and row.DAYS_CREDIT_ENDDATE < 0:
        early_shutdown = 1
    return early_shutdown


def quasi_constant(data,threshold = 0.98):
    # create empty list
    quasi_constant_feature = []
              
    # loop over all the columns
    for feature in data.columns:
    
         # calculate the ratio.
        predominant = (data[feature].value_counts() / np.float(len(data))).sort_values(ascending=False).values[0]
          
         # append the column name if it is bigger than the threshold
        if predominant >= threshold:
             quasi_constant_feature.append(feature)   
 
    return quasi_constant_feature

# Check for duplicate feature in data
def duplicated_features(data):
      duplicated_feat = []
    
      for i in range(0,len(data.columns)):
            if i % 10 == 0: # will show the loop
                print(i)
            col_1 = data.columns[i]
            
            for col_2 in data.columns[i + 1:]:
                if(data[col_1].equals(data[col_2])):
                    duplicated_feat.append(col_2) 
      
      return duplicated_feat

# find and remove correlated features
def correlation(data, threshold):
        col_corr = set()  # Set of all the names of correlated columns
        corr_matrix = data.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i]  # getting the name of column
                    col_corr.add(colname)
        return col_corr    



In [4]:
def buro_add_feature(df_breau):
    df_bureau_new = pd.DataFrame()
    # kredi başvuru sayısı
    df_bureau_new["BURO_CREDIT_APPLICATION_COUNT"] = df_breau.groupby("SK_ID_CURR").count()["SK_ID_BUREAU"]

    # aktif kredi sayısı
    df_bureau_new["BURO_ACTIVE_CREDIT_APPLICATION_COUNT"] = \
        df_breau[df_breau["CREDIT_ACTIVE"] == "Active"].groupby("SK_ID_CURR").count()["CREDIT_ACTIVE"]
    df_bureau_new["BURO_ACTIVE_CREDIT_APPLICATION_COUNT"].fillna(0, inplace=True)

    # pasif kredi sayısı
    df_bureau_new["BURO_CLOSED_CREDIT_APPLICATION_COUNT"] = \
        df_breau[df_breau["CREDIT_ACTIVE"] == "Closed"].groupby("SK_ID_CURR").count()["CREDIT_ACTIVE"]
    df_bureau_new["BURO_CLOSED_CREDIT_APPLICATION_COUNT"].fillna(0, inplace=True)

    # erken kredi kapama
    df_bureau_new["BURO_EARLY_SHUTDOWN_NEW"] = df_breau.apply(lambda x: feature_early_shutdown(x), axis=1)

    # geciktirilmiş ödeme sayısı
    df_bureau_new["BURO_NUMBER_OF_DELAYED_PAYMENTS"] = \
        df_breau[df_breau["AMT_CREDIT_MAX_OVERDUE"] != 0].groupby("SK_ID_CURR")["AMT_CREDIT_MAX_OVERDUE"].count()
    df_bureau_new["BURO_NUMBER_OF_DELAYED_PAYMENTS"].fillna(0, inplace=True)

    # son kapanmış başvurusu üzerinden geçen max süre
    df_bureau_new["BURO_MAX_TIME_PASSED_CREDIT_APPLICATION"] = \
        df_breau[df_breau["CREDIT_ACTIVE"] == "Closed"].groupby("SK_ID_CURR")["DAYS_ENDDATE_FACT"].max()
    df_bureau_new["BURO_MAX_TIME_PASSED_CREDIT_APPLICATION"].fillna(0, inplace=True)

    # geciktirilmiş max ödeme tutari
    df_bureau_new["BURO_MAX_DELAYED_PAYMENTS"] = df_breau.groupby("SK_ID_CURR")["AMT_CREDIT_MAX_OVERDUE"].max()
    df_bureau_new["BURO_MAX_DELAYED_PAYMENTS"].fillna(0, inplace=True)

    # geciktirilmiş ödeyenlerden oluşan top liste - en yüksek 100
    # gecikme olan (80302, 12)
    df_bureau_new["BURO_DELAYED_PAYMENTS_TOP_100_NEW"] = \
        df_bureau_new.sort_values("BURO_MAX_DELAYED_PAYMENTS", ascending=False)["BURO_MAX_DELAYED_PAYMENTS"].rank()
    df_bureau_new["BURO_DELAYED_PAYMENTS_TOP_100_NEW"].fillna(0, inplace=True)

    # kredi uzatma yapilmis mi
    df_bureau_new["BURO_IS_CREDIT_EXTENSION_NEW"] = df_breau.groupby("SK_ID_CURR")["CNT_CREDIT_PROLONG"].count().apply(
        lambda x: 1 if x > 0 else 0)

    # max yapilan kredi uzatmasi
    df_bureau_new["BURO_CREDIT_EXTENSION_MAX"] = df_breau.groupby("SK_ID_CURR")["CNT_CREDIT_PROLONG"].max()
    df_bureau_new["BURO_CREDIT_EXTENSION_MAX"].fillna(0, inplace=True)

    # unsuccessful credit payment - borç takarak kapanmış kredi ödemeleri tespit et
    df_bureau_new["BURO_IS_UNSUCCESSFUL_CREDIT_PAYMENT_NEW"] = \
        df_breau[(df_breau["CREDIT_ACTIVE"] == "Closed") & (df_breau["AMT_CREDIT_SUM_DEBT"] > 0)].groupby(
            "SK_ID_CURR").all()["AMT_CREDIT_SUM_DEBT"].apply(lambda x: 1 if x == True else 0)
    df_bureau_new["BURO_IS_UNSUCCESSFUL_CREDIT_PAYMENT_NEW"].fillna(0, inplace=True)

    return df_bureau_new


def load_data_with_application_train(num_rows=None):
    df_app_train = application_train()
    print("application_train df shape:", df_app_train.shape)
    bureau, bureau_add_features = bureau_and_balance()
    print("Bureau df shape:", bureau.shape)
    bureau = bureau.fillna(0)
    return df_app_train, bureau, bureau_add_features


def load_data_only_bureau_and_bureau_balance(num_rows=None):
    bureau, bureau_add_features = bureau_and_balance()
    print("Bureau df shape:", bureau.shape)
    bureau = bureau.fillna(0)
    return bureau, bureau_add_features


def app_train_bureau_merge(num_rows=None):
    df_app_train, bureau, bureau_add_features = load_data_with_application_train(num_rows)
    # df_merge = pd.merge(df_app_train, bureau, on=['SK_ID_CURR'],how='inner')
    df_merge = bureau
    # print("app_train, bureau merge shape:", df_merge.shape)
    print("bureau merge shape:", df_merge.shape)
    df_final = pd.merge(df_merge, bureau_add_features, on=['SK_ID_CURR'], how='inner')
    print("Bureau add features df shape:", bureau_add_features.shape)
    del df_app_train, bureau, bureau_add_features, df_merge
    gc.collect()
    return df_final


def bureau_and_bureau_balance_features(num_rows=None):
    bureau, bureau_add_features = load_data_only_bureau_and_bureau_balance(num_rows)
    df_final = pd.merge(bureau, bureau_add_features, on=['SK_ID_CURR'], how='inner')
    print("Bureau add features df shape:", bureau_add_features.shape)
    del bureau, bureau_add_features
    gc.collect()
    return df_final


def application_train():
    conn = pymysql.connect(host='35.228.28.142', port=int(63306), user='group2', passwd='123654', db='home_credit')
    df_app_train = pd.read_sql_query("SELECT * FROM application_train", conn)
    df_app_train = df_app_train[["TARGET", "SK_ID_CURR"]]
    # df_app_train = df_app_train.dropna()
    df_app_train.reset_index(drop=True, inplace=True)
    gc.collect()
    return df_app_train


# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(nan_as_category=True):
    conn = pymysql.connect(host='35.228.28.142', port=int(63306), user='group2', passwd='123654', db='home_credit')
    bureau = pd.read_sql_query("SELECT * FROM bureau", conn)
    bb = pd.read_sql_query("SELECT * FROM bureau_balance", conn)
    bureau["AMT_CREDIT_SUM_DEBT"] = bureau["AMT_CREDIT_SUM_DEBT"].fillna(0)
    bureau.fillna(0, inplace=True)
    bb.fillna(0, inplace=True)
    # bureau = bureau.dropna()
    bureau.reset_index(drop=True, inplace=True)
    # bb = bb.dropna()
    bb.reset_index(drop=True, inplace=True)

    # add_features
    bureau_add_features = buro_add_feature(df_breau=bureau)

    # sum agg b_balance
    # Status_sum ile ilgili yeni bir degisken olusturma
    bb_dummy = pd.get_dummies(bb, dummy_na=True)
    agg_list = {"MONTHS_BALANCE": "count",
                "STATUS_0": ["sum"],
                "STATUS_1": ["sum"],
                "STATUS_2": ["sum"],
                "STATUS_3": ["sum"],
                "STATUS_4": ["sum"],
                "STATUS_5": ["sum"],
                "STATUS_C": ["sum"],
                "STATUS_X": ["sum"]}
    bb_sum_agg = bb_dummy.groupby("SK_ID_BUREAU").agg(agg_list)
    # Degisken isimlerinin yeniden adlandirilmasi
    bb_sum_agg.columns = pd.Index(["BURO_" + col[0] + "_" + col[1].upper() for col in bb_sum_agg.columns.tolist()])
    # Status_sum ile ilgili yeni bir degisken olusturma
    bb_sum_agg['BURO_NEW_STATUS_SCORE'] = bb_sum_agg['BURO_STATUS_1_SUM'] + bb_sum_agg['BURO_STATUS_2_SUM'] ^ 2 + \
                                          bb_sum_agg['BURO_STATUS_3_SUM'] ^ 3 + bb_sum_agg['BURO_STATUS_4_SUM'] ^ 4 + \
                                          bb_sum_agg['BURO_STATUS_5_SUM'] ^ 5
    bb_sum_agg.drop(
        ['BURO_STATUS_1_SUM', 'BURO_STATUS_2_SUM', 'BURO_STATUS_3_SUM', 'BURO_STATUS_4_SUM', 'BURO_STATUS_5_SUM'],
        axis=1, inplace=True)

    # CREDIT_TYPE degiskeninin sinif sayisini 3'e düsürmek
    bureau['CREDIT_TYPE'] = bureau['CREDIT_TYPE'].replace(['Car loan',
                                                           'Mortgage',
                                                           'Microloan',
                                                           'Loan for business development',
                                                           'Another type of loan',
                                                           'Unknown type of loan',
                                                           'Loan for working capital replenishment',
                                                           "Loan for purchase of shares (margin lending)",
                                                           'Cash loan (non-earmarked)',
                                                           'Real estate loan',
                                                           "Loan for the purchase of equipment",
                                                           "Interbank credit",
                                                           "Mobile operator loan"], 'Rare')

    # CREDIT_ACTIVE degiskeninin sinif sayisini 2'ye düsürmek (Sold' u Closed a dahil etmek daha mi uygun olur ???)
    bureau['CREDIT_ACTIVE'] = bureau['CREDIT_ACTIVE'].replace(['Bad debt', 'Sold'], 'Active')

    # one hot encoding start
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    # one hot encoding end

    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size'],
                       "STATUS_0": ["mean"],
                       "STATUS_C": ["mean"],
                       "STATUS_X": ["mean"]}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']

    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

    # b_balance sum değşkenlerinin eklenmesi
    bb_agg["BURO_MONTHS_BALANCE_COUNT"] = bb_sum_agg["BURO_MONTHS_BALANCE_COUNT"]
    bb_agg["BURO_STATUS_0_SUM"] = bb_sum_agg["BURO_STATUS_0_SUM"]
    bb_agg["BURO_STATUS_C_SUM"] = bb_sum_agg["BURO_STATUS_C_SUM"]
    bb_agg["BURO_STATUS_X_SUM"] = bb_sum_agg["BURO_STATUS_X_SUM"]
    bb_agg["BURO_NEW_STATUS_SCORE"] = bb_sum_agg["BURO_NEW_STATUS_SCORE"]

    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')

    bureau["BURO_MONTHS_BALANCE_COUNT"].fillna(0, inplace=True)
    bureau["BURO_STATUS_0_SUM"].fillna(0, inplace=True)
    bureau["BURO_STATUS_C_SUM"].fillna(0, inplace=True)
    bureau["BURO_STATUS_X_SUM"].fillna(0, inplace=True)
    bureau["BURO_NEW_STATUS_SCORE"].fillna(0, inplace=True)

    ##ek son değişkenler
    # ortalama kac aylık kredi aldıgını gösteren yeni degisken
    bureau["BURO_NEW_MONTHS_CREDIT"] = round((bureau.DAYS_CREDIT_ENDDATE - bureau.DAYS_CREDIT) / 30)

    bureau.drop(columns='SK_ID_BUREAU', inplace=True)

    del bb, bb_agg
    gc.collect()

    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum', 'std'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum', 'std', 'median'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'DAYS_CREDIT_UPDATE': ['min', 'max', 'mean'],
        'AMT_ANNUITY': ['max', 'mean'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']

    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['BURO_ACT_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['BURO_CLS_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg, bureau_add_features


def application_train_g():
    print("application_tarin_g")
    conn = pymysql.connect(host='35.228.28.142', port=int(63306), user='group2', passwd='123654', db='home_credit')
    df = pd.read_sql_query("SELECT * FROM application_train", conn)
    test_df = pd.read_sql_query("SELECT * FROM application_test", conn)
    print("connection finished")
    df = reduce_mem_usage(df)
    test_df = reduce_mem_usage(test_df)

    df = df.append(test_df).reset_index()

    pd.set_option('display.max_columns', 500)
    pd.set_option('display.max_rows', 500)

    le = LabelEncoder()

    df["NAME_EDUCATION_TYPE"] = le.fit_transform(df["NAME_EDUCATION_TYPE"])
    df.loc[(df["NAME_EDUCATION_TYPE"] == 1), "NAME_EDUCATION_TYPE"] = 0

    df.loc[(df["CNT_FAM_MEMBERS"] > 3), "CNT_FAM_MEMBERS"] = 4

    df = df[df['CODE_GENDER'] != 'XNA']

    lbe = LabelEncoder()

    for col in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[col] = lbe.fit_transform(df[col])

    # df = pd.get_dummies(df, dummy_na = True)

    nom_list = [
        'EMERGENCYSTATE_MODE',
        'FONDKAPREMONT_MODE',
        'HOUSETYPE_MODE',
        'NAME_CONTRACT_TYPE',
        'NAME_FAMILY_STATUS',
        'NAME_HOUSING_TYPE',
        'NAME_INCOME_TYPE',
        'NAME_TYPE_SUITE',
        'OCCUPATION_TYPE',
        'ORGANIZATION_TYPE',
        'WALLSMATERIAL_MODE',
        'WEEKDAY_APPR_PROCESS_START']

    df = rare_encoding(df, nom_list)
    df = pd.get_dummies(df, columns=nom_list, drop_first=True)

    # new_features
    # 1
    df["NEW_GOODSPRICE/CREDIT"] = df["AMT_GOODS_PRICE"] / df["AMT_CREDIT"]
    # 2
    df["NEW_ANNUITY/CREDIT"] = (df["AMT_ANNUITY"] / df["AMT_CREDIT"])
    # 3
    df["NEW_INCOME/ANNUITY"] = df["AMT_INCOME_TOTAL"] / df["AMT_ANNUITY"]
    # 4
    df["NEW_DAYS_LAST_PHONE_CHANGE"] = df["DAYS_LAST_PHONE_CHANGE"]
    df.loc[(df["NEW_DAYS_LAST_PHONE_CHANGE"] == 0), "NEW_DAYS_LAST_PHONE_CHANGE"] = 1
    df.loc[(df["NEW_DAYS_LAST_PHONE_CHANGE"] != 0), "NEW_DAYS_LAST_PHONE_CHANGE"] = 0
    # 5
    df["DAYS_BIRTH"] = df["DAYS_BIRTH"] / 365
    df["NEW_DAYS_BIRTH"] = df["DAYS_BIRTH"]
    df.loc[(df["NEW_DAYS_BIRTH"] <= 41), "NEW_DAYS_BIRTH"] = 1
    df.loc[(df["NEW_DAYS_BIRTH"] > 41), "NEW_DAYS_BIRTH"] = 0
    # 6
    df["NEW_CREDIT/INCOME"] = df["AMT_CREDIT"] / df["AMT_INCOME_TOTAL"]
    # 7
    df["NEW_WORK/NOTWORK"] = df["DAYS_EMPLOYED"]
    df.loc[(df["NEW_WORK/NOTWORK"] == 0), "NEW_WORK/NOTWORK"] = 0  # ÇALIŞMAYANLAR
    df.loc[(df["NEW_WORK/NOTWORK"] != 0), "NEW_WORK/NOTWORK"] = 1  # ÇALIŞANLAR
    # 8
    df["NEW_INCOME/CREDIT"] = df["AMT_INCOME_TOTAL"] / df["AMT_CREDIT"]
    # 9
    # En yakın zaman (soruşturma olmayan 0, saat+gün+ hafta+ay için 1, ay+yıl için 2)
    df["NEW_REQ"] = df["AMT_REQ_CREDIT_BUREAU_WEEK"]
    # yakın ve orta zamanda soruşturma
    df.loc[(df["AMT_REQ_CREDIT_BUREAU_HOUR"] > 0), "NEW_REQ"] = 1
    df.loc[(df["AMT_REQ_CREDIT_BUREAU_DAY"] > 0), "NEW_REQ"] = 1

    df.loc[(df["AMT_REQ_CREDIT_BUREAU_HOUR"] == 0) & (df["AMT_REQ_CREDIT_BUREAU_DAY"] == 0) & (
            df["AMT_REQ_CREDIT_BUREAU_WEEK"] > 0), "NEW_REQ"] = 1
    df.loc[(df["AMT_REQ_CREDIT_BUREAU_HOUR"] == 0) & (df["AMT_REQ_CREDIT_BUREAU_DAY"] == 0) & (
            df["AMT_REQ_CREDIT_BUREAU_MON"] > 0), "NEW_REQ"] = 1
    # uzak zaman soruşturma
    df.loc[(df["AMT_REQ_CREDIT_BUREAU_HOUR"] == 0) & (df["AMT_REQ_CREDIT_BUREAU_DAY"] == 0) &
           (df["AMT_REQ_CREDIT_BUREAU_WEEK"] == 0) & (df["AMT_REQ_CREDIT_BUREAU_MON"] == 0) &
           (df["AMT_REQ_CREDIT_BUREAU_QRT"] > 0), "NEW_REQ"] = 2

    df.loc[(df["AMT_REQ_CREDIT_BUREAU_HOUR"] == 0) & (df["AMT_REQ_CREDIT_BUREAU_DAY"] == 0) &
           (df["AMT_REQ_CREDIT_BUREAU_WEEK"] == 0) & (df["AMT_REQ_CREDIT_BUREAU_MON"] == 0) &
           (df["AMT_REQ_CREDIT_BUREAU_YEAR"] > 0), "NEW_REQ"] = 2
    # soruşturma olmayanlar
    df.loc[(pd.isna(df["NEW_REQ"])), "NEW_REQ"] = 0

    # eski grup yeni feature ları
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    df['NEW_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

    df['NEW_EXT_RESOURCE_3_CREDIT_TO_GOODS_RATIO'] = df['EXT_SOURCE_3'] / (df['AMT_CREDIT'] / df['AMT_GOODS_PRICE'])
    df['NEW_EXT_RESOURCE_2_CREDIT_TO_GOODS_RATIO'] = df['EXT_SOURCE_2'] / (df['AMT_CREDIT'] / df['AMT_GOODS_PRICE'])
    df['NEW_EXT_RESOURCE_1_CREDIT_TO_GOODS_RATIO'] = df['EXT_SOURCE_1'] / (df['AMT_CREDIT'] / df['AMT_GOODS_PRICE'])
    

    df.drop("index", axis=1, inplace=True)

    df.columns = pd.Index(["APP_" + col for col in df.columns.tolist()])

    df.rename(columns={"APP_SK_ID_CURR": "SK_ID_CURR"}, inplace=True)

    df.rename(columns={"APP_TARGET": "TARGET"}, inplace=True)
    
    print("DF SAYISI:")
    print(len(df))

    return df


def previous_application():
    conn = pymysql.connect(host='35.228.28.142', port=int(63306), user='group2', passwd='123654', db='home_credit')
    df_prev = pd.read_sql_query("SELECT * FROM previous_application", conn)
    df_prev = reduce_mem_usage(df_prev)
  

    # Features that has outliers
    feat_outlier = ["AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT", "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE",
                    "SELLERPLACE_AREA"]

    # Replacing the outliers of the features with their own upper values
    for var in feat_outlier:
        Q1 = df_prev[var].quantile(0.01)
        Q3 = df_prev[var].quantile(0.99)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        df_prev[var][(df_prev[var] > upper)] = upper

    # 365243 value will be replaced by NaN in the following features
    feature_replace = ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE',
                       'DAYS_TERMINATION']

    for var in feature_replace:
        df_prev[var].replace(365243, np.nan, inplace=True)

    # One hot encoding
    categorical_columns = [col for col in df_prev.columns if df_prev[col].dtype == 'object']
    #df_prev = pd.get_dummies(df_prev, columns=categorical_columns, dummy_na=True)

    df_prev = df_prev.drop(categorical_columns, axis = 1) 
    
    # Creating new features

    df_prev['APP_CREDIT_PERC'] = df_prev['AMT_APPLICATION'] / df_prev['AMT_CREDIT']
    df_prev['NEW_CREDIT_TO_ANNUITY_RATIO'] = df_prev['AMT_CREDIT'] / df_prev['AMT_ANNUITY']
    df_prev['NEW_DOWN_PAYMENT_TO_CREDIT'] = df_prev['AMT_DOWN_PAYMENT'] / df_prev['AMT_CREDIT']
    df_prev['NEW_TOTAL_PAYMENT'] = df_prev['AMT_ANNUITY'] * df_prev['CNT_PAYMENT']
    df_prev['NEW_TOTAL_PAYMENT_TO_AMT_CREDIT'] = df_prev['NEW_TOTAL_PAYMENT'] / df_prev['AMT_CREDIT']
    # Innterest ratio previous application (simplified)

    df_prev['SIMPLE_INTERESTS'] = (df_prev['NEW_TOTAL_PAYMENT'] / df_prev['AMT_CREDIT'] - 1) / df_prev['CNT_PAYMENT']

    # Previous applications numeric features
    num_aggregations = {}
    num_cols = df_prev.select_dtypes(exclude=['object'])
    num_cols.drop(['SK_ID_PREV', 'SK_ID_CURR'], axis=1, inplace=True)

    for num in num_cols:
        num_aggregations[num] = ['min', 'max', 'mean', 'var', 'sum']

        # Previous applications categoric features
    cat_aggregations = {}
    for i in df_prev.columns:
        if df_prev[i].dtypes == "O":
            cat_aggregations[i] = ['mean']

    prev_agg = df_prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

    # Dropping features with small variance
    features_with_small_variance = prev_agg.columns[(prev_agg.std(axis=0) < .1).values]
    prev_agg.drop(features_with_small_variance, axis=1, inplace=True)
    prev_agg.reset_index(inplace=True)

    return prev_agg


def previous_application_b():
    conn = pymysql.connect(host='35.228.28.142', port=int(63306), user='group2', passwd='123654', db='home_credit')
    df_control=reduce_mem_usage(pd.read_sql_query('SELECT * FROM home_credit.previous_application',conn))
    
    #en son yapilan basvuruya göre filtreledik
    df_days=df_control.loc[df_control.groupby('SK_ID_CURR')['DAYS_DECISION'].idxmax()]
    df_days = df_days[['SK_ID_CURR','DAYS_DECISION','NAME_CONTRACT_STATUS']]
    #LAST_APPEAL kolonu olarak degistirdik aslinda buna gerek yok feature importance da da yan yanalar gene de koydum
    df_days=df_days.rename(columns={'DAYS_DECISION': 'LAST_APPEAL','NAME_CONTRACT_STATUS': 'STATUS_OF_LAST_APPEAL'})
    AMOUNT_OF_APPEAL = [] 
    AMOUNT_OF_APPEAL=df_control.groupby('SK_ID_CURR')['NAME_CONTRACT_STATUS'].apply(lambda y: (y=='Approved').sum()).reset_index(name='count')
    AMOUNT_OF_APPEAL=AMOUNT_OF_APPEAL.rename(columns={'count': 'AMT_APPROVED'})
    AMOUNT_OF_TOTAL_APPEAL=df_control.groupby(['SK_ID_CURR']).count().sort_values('SK_ID_PREV', ascending=False)  
    AMOUNT_OF_TOTAL_APPEAL=AMOUNT_OF_TOTAL_APPEAL.reset_index().sort_values(by='SK_ID_CURR', ascending=True)
    AMOUNT_OF_TOTAL_APPEAL=AMOUNT_OF_TOTAL_APPEAL.rename(columns={'SK_ID_PREV': 'AMOUNT_OF_TOTAL_APPEAL'})
    AMOUNT_OF_TOTAL_APPEAL = AMOUNT_OF_TOTAL_APPEAL[['SK_ID_CURR', 'AMOUNT_OF_TOTAL_APPEAL']]
    AMOUNT_OF_mean_total=df_control.groupby('SK_ID_CURR')['AMT_APPLICATION'].sum().reset_index(name='count')
    AMOUNT_OF_mean_approved=df_control.groupby('SK_ID_CURR')['AMT_CREDIT'].sum().reset_index(name='count') 
    df_control_last=df_control.sort_values(by='SK_ID_CURR', ascending=True)
    df_control_last=df_control_last.drop_duplicates(subset=['SK_ID_CURR'])
    
    
    
    
    AMOUNT_OF_mean_total=AMOUNT_OF_mean_total.rename(columns={'count': 'AMOUNT_OF_mean_total'})
    AMOUNT_OF_mean_approved=AMOUNT_OF_mean_approved.rename(columns={'count': 'AMOUNT_OF_mean_approved'})
    
    ############################################################
    
    df_result = pd.merge(AMOUNT_OF_mean_approved, AMOUNT_OF_mean_total, on='SK_ID_CURR') 

    df_result = pd.merge(df_result, AMOUNT_OF_TOTAL_APPEAL, on='SK_ID_CURR') 
    
    df_result = pd.merge(df_result, AMOUNT_OF_APPEAL, on='SK_ID_CURR') 
    
    df_result = pd.merge(df_result, df_days, on='SK_ID_CURR') 
    
    df_result['APPROVED_CREDIT/TOTAL_CREDIT'] = df_result['AMOUNT_OF_mean_approved']/df_result['AMOUNT_OF_mean_total'] 
    
    df_result['AMT_APPROVED/TOTAL'] = df_result['AMT_APPROVED']/df_result['AMOUNT_OF_TOTAL_APPEAL'] 
    #burada app train den veri cekmek zorundayiz ! veri cektigim sorgu calismayabilir, alttaki satir yani
    
    
   ############################################################
    
    
    print("Credit Application")
    df_target= reduce_mem_usage(pd.read_sql_query('SELECT * FROM home_credit.application_train',conn))
    print("Credit App bağlandı")

    df_target = df_target[['SK_ID_CURR', 'TARGET']]
    df_ml = pd.merge(df_result, df_target,how='right', on='SK_ID_CURR')
    df_ml = pd.merge(df_ml, df_control_last,how='left', on='SK_ID_CURR')
    
    print("installments_payments")
    df_install = reduce_mem_usage(pd.read_sql_query('SELECT * FROM home_credit.installments_payments',conn))
    print("installments_payments bağlandı")

    df_install['Paying_Rate'] = df_install['AMT_PAYMENT']/df_install['AMT_INSTALMENT']
    df_install['Delaying_Number'] = df_install['DAYS_ENTRY_PAYMENT']-df_install['DAYS_INSTALMENT']
    df_install=df_install.drop_duplicates(subset=['SK_ID_CURR'])
    df_ml = pd.merge(df_ml, df_install,how='left', on='SK_ID_CURR')
    
    df_ml_verison2=df_ml.replace('XAP',np.NaN)
    df_ml_verison2=df_ml_verison2.replace('XNA',np.NaN)
    df_ml_verison2=df_ml_verison2.replace('XAP',np.NaN)
    df_ml_verison3 = df_ml_verison2.dropna(subset=['NAME_CONTRACT_TYPE','APPROVED_CREDIT/TOTAL_CREDIT','NAME_CLIENT_TYPE','PRODUCT_COMBINATION','CHANNEL_TYPE'])

    
    def rare_encoding_sum(df, variable,threshold):

        list1=[]
        for var in df[variable].unique(): 
            list1.append(var) 
        for x in range(len(list1)): 
            if df[variable].value_counts(dropna=False)[list1[x]]*100/len(df[variable]) <= threshold :
                    df[variable].replace({list1[x]: "Rare"}, inplace=True)

                    
    
     
        
    # cok fazla kategoriler icin yuzde5 rare kategori icine toplayabildiklerimizi topladik
    rare_encoding_sum(df_ml_verison3,'PRODUCT_COMBINATION',1.8)
    rare_encoding_sum(df_ml_verison3,'NAME_YIELD_GROUP',2)
    rare_encoding_sum(df_ml_verison3,'NAME_SELLER_INDUSTRY',2.14)
    rare_encoding_sum(df_ml_verison3,'NAME_GOODS_CATEGORY',1.4) 
    rare_encoding_sum(df_ml_verison3,'CODE_REJECT_REASON',2.5)
    rare_encoding_sum(df_ml_verison3,'NAME_CASH_LOAN_PURPOSE',1.5)
    rare_encoding_sum(df_ml_verison3,'NAME_PAYMENT_TYPE',1)
  
    #lambda x: x*10 if x<2 else (x**2 if x<4 else x+10)
    df_ml_verison3['Delaying_Number-CAT'] = df_ml_verison3['Delaying_Number'].apply(lambda x:'Paid Earlier' if x < 0 else ( 'Paid in Time' if x == 0 else 'Paid Later') )
    df_ml_verison3=df_ml_verison3.drop("AMT_APPROVED", axis=1)
    df_ml_verison3=df_ml_verison3.drop("AMOUNT_OF_mean_approved", axis=1)
    df_ml_verison3['started_on_time'] = df_ml_verison3['DAYS_FIRST_DUE']-df_ml_verison3['DAYS_LAST_DUE_1ST_VERSION']
    df_ml_verison3['started_on_time-CAT'] = df_ml_verison3['started_on_time'].apply(lambda x:'Paid Later' if x < 0 else ( 'Paid in Time' if x == 0 else 'Paid Earlier') )
    df_ml_verison3['finished_on_time'] = df_ml_verison3['DAYS_LAST_DUE']-df_ml_verison3['DAYS_TERMINATION']
    df_ml_verison3['finished_on_time-CAT'] = df_ml_verison3['finished_on_time'].apply(lambda x:'Paid Later' if x < 0 else ( 'Paid in Time' if x == 0 else 'Paid Earlier') )
    #gereksiz featurelar atiliyor
    df_ml_verison3=df_ml_verison3.drop("started_on_time", axis=1)
    df_ml_verison3=df_ml_verison3.drop("finished_on_time", axis=1)
    df_ml_verison3=df_ml_verison3.drop("DAYS_FIRST_DUE", axis=1)
    df_ml_verison3=df_ml_verison3.drop("DAYS_LAST_DUE_1ST_VERSION", axis=1)
    df_ml_verison3=df_ml_verison3.drop("DAYS_LAST_DUE", axis=1)
    df_ml_verison3=df_ml_verison3.drop("DAYS_TERMINATION", axis=1)
    #df_ml_verison3=df_ml_verison3.drop("SK_ID_CURR", axis=1)
    df_ml_verison3=df_ml_verison3.drop("TARGET", axis=1)
    df_ml_verison3=df_ml_verison3.drop("SK_ID_PREV_x", axis=1)
    df_ml_verison3=df_ml_verison3.drop("SK_ID_PREV_y", axis=1)
    #sayilarda cok ufak bir nan cikti modele sokabilmek icin atiyorum
    df_ml_verison3=df_ml_verison3.dropna(subset=['Delaying_Number', 'Paying_Rate','AMT_PAYMENT','DAYS_ENTRY_PAYMENT','DAYS_INSTALMENT','NUM_INSTALMENT_NUMBER','NUM_INSTALMENT_VERSION'])
    cols_to_use=[]
    for col in df_ml_verison3.columns: 
        cols_to_use.append(col) 


    #kategoriklerde nan value lari atmak istemiyorum missing koydum 
    # Now we impute the missing values with SimpleImputer

    # create an instance of the simple imputer
    # we indicate that we want to impute by replacing NA
    # with 'Missing'

    imputer = SimpleImputer(strategy='constant', fill_value = 'Missing')

    # we fit the imputer to the train set
    # the imputer will learn the median of all variables
    imputer.fit(df_ml_verison3)   # SIL
    #has prev appl
    #not prev apl
    df_ml_verison3=imputer.transform(df_ml_verison3)
    df_ml_verison3 = pd.DataFrame(df_ml_verison3, columns=cols_to_use)
    df_ml_verison3=df_ml_verison3.replace([np.inf, -np.inf], 0)
    
    df_ml_verison3['Paying_Rate'] = df_ml_verison3['Paying_Rate'].fillna(0)

    nom_list=['FLAG_LAST_APPL_PER_CONTRACT','NAME_CASH_LOAN_PURPOSE','NAME_SELLER_INDUSTRY','PRODUCT_COMBINATION','WEEKDAY_APPR_PROCESS_START','CHANNEL_TYPE',
     'NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_CONTRACT_TYPE','NAME_PAYMENT_TYPE','NAME_TYPE_SUITE']
    df_ml_verison3=pd.get_dummies(df_ml_verison3, columns=nom_list, drop_first=True)
    ord_list_V3=['STATUS_OF_LAST_APPEAL', 'NAME_CONTRACT_STATUS', 'NAME_YIELD_GROUP', 'finished_on_time-CAT','Delaying_Number-CAT' ,'started_on_time-CAT', 'NAME_PRODUCT_TYPE' ,'CODE_REJECT_REASON', 'NAME_CLIENT_TYPE']

    for i in ord_list_V3:
        labelencoder = LabelEncoder()
        le = LabelEncoder()
        df_ml_verison3[i] = le.fit_transform(df_ml_verison3[i])

    
    #alinacak featurelar yukaridadir
    gc.collect()
    return df_ml_verison3


def credit_card_balance():
    conn = pymysql.connect(host='35.228.28.142', port=int(63306), user='group2', passwd='123654', db='home_credit')
    ccb = pd.read_sql_query("SELECT * FROM credit_card_balance", conn)

    ccb = reduce_mem_usage(ccb)

    ccb = ccb.groupby('SK_ID_CURR').agg(['mean'])
    e = 0
    ccb.columns = pd.Index(
        ['CC_' + ccb.columns[e][0] + "_" + ccb.columns[e][1].upper() for e in range(ccb.columns.size)])

    # new feature1: calculating the rate of balance(loan) to the credit card limit
    ccb["CC_NEW_LOAN_TO_CREDIT_LIMIT_RATE"] = (ccb["CC_AMT_BALANCE_MEAN"] + 1) / (
            ccb["CC_AMT_CREDIT_LIMIT_ACTUAL_MEAN"] + 1)

    # new feature2: at what rate the customer paid the loan:CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN /
    # CC_AMT_TOTAL_RECEIVABLE_MEAN: CC_PAID_AMOUNT_RATE
    ccb["CC_NEW_PAID_AMOUNT_RATE"] = (ccb["CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN"] + 1) / (
            ccb["CC_AMT_TOTAL_RECEIVABLE_MEAN"] + 1) * 100

    # new feature3: how much money the customer withdrew in avg from ATM per drawing:AMOUNT PER ATM DRAWING
    ccb["CC_NEW_AMT_PER_ATM_DRAWING_MEAN"] = (ccb["CC_AMT_DRAWINGS_ATM_CURRENT_MEAN"] + 1) / (
            ccb["CC_CNT_DRAWINGS_ATM_CURRENT_MEAN"] + 1)

    # new feature4: how much money the customer withdrew from POS in avg per drawing:AMOUNT PER POS DRAWING
    ccb["CC_NEW_AMT_PER_POS_DRAWING_MEAN"] = (ccb["CC_AMT_DRAWINGS_POS_CURRENT_MEAN"] + 1) / (
            ccb["CC_CNT_DRAWINGS_POS_CURRENT_MEAN"] + 1)

    ccb = pd.concat([ccb.loc[:, "CC_NEW_LOAN_TO_CREDIT_LIMIT_RATE"],
                     ccb.loc[:, "CC_NEW_PAID_AMOUNT_RATE"],
                     ccb.loc[:, "CC_AMT_CREDIT_LIMIT_ACTUAL_MEAN"],
                     ccb.loc[:, "CC_AMT_PAYMENT_CURRENT_MEAN"],
                     ccb.loc[:, "CC_MONTHS_BALANCE_MEAN"],
                     ccb.loc[:, "CC_CNT_INSTALMENT_MATURE_CUM_MEAN"],
                     ccb.loc[:, "CC_AMT_INST_MIN_REGULARITY_MEAN"],
                     ccb.loc[:, "CC_AMT_DRAWINGS_ATM_CURRENT_MEAN"],
                     ccb.loc[:, "CC_AMT_DRAWINGS_POS_CURRENT_MEAN"],
                     ccb.loc[:, "CC_CNT_DRAWINGS_ATM_CURRENT_MEAN"],
                     ccb.loc[:, "CC_CNT_DRAWINGS_POS_CURRENT_MEAN"],
                     ccb.loc[:, "CC_NEW_AMT_PER_ATM_DRAWING_MEAN"],
                     ccb.loc[:, "CC_NEW_AMT_PER_POS_DRAWING_MEAN"]], axis=1)

    return ccb


def prepare_instalment_payment():
    conn = pymysql.connect(host='35.228.28.142', port=int(63306), user='group2', passwd='123654', db='home_credit')
    df_installments_payments = pd.read_sql_query("SELECT * FROM installments_payments", conn)

    df_installments_payments = reduce_mem_usage(df_installments_payments)

    # O anki taksitin yuzde kaci odendi
    df_installments_payments[['AMT_PAYMENT']] = df_installments_payments[['AMT_PAYMENT']].fillna(value=0)
    df_installments_payments['NEW_INSTALMENT_PAYMENT_RATE'] = df_installments_payments['AMT_PAYMENT'] / \
                                                              df_installments_payments['AMT_INSTALMENT'] * 100

    # O anki taksit son odeme gununden kac gun once odenmis. Bu degisken "NEW_INSTALMENT_PAYMENT_STATUS" degerini bulabilmek icin gecici olusturulur.
    df_installments_payments['NEW_DAY_BEFORE_END_DATE'] = df_installments_payments['DAYS_INSTALMENT'] - \
                                                          df_installments_payments['DAYS_ENTRY_PAYMENT']

    df_installments_payments["NEW_INSTALMENT_PAYMENT_STATUS"] = "No Payment"
    df_installments_payments.loc[
        df_installments_payments['NEW_DAY_BEFORE_END_DATE'] == 0, "NEW_INSTALMENT_PAYMENT_STATUS"] = "In Time"
    df_installments_payments.loc[
        df_installments_payments['NEW_DAY_BEFORE_END_DATE'] > 0, "NEW_INSTALMENT_PAYMENT_STATUS"] = "Early"
    df_installments_payments.loc[
        df_installments_payments['NEW_DAY_BEFORE_END_DATE'] < 0, "NEW_INSTALMENT_PAYMENT_STATUS"] = "Late"

    df_installments_payments["NEW_INS_IS_LATE"] = "No"
    df_installments_payments.loc[df_installments_payments['NEW_DAY_BEFORE_END_DATE'] < 0, "NEW_INS_IS_LATE"] = "Yes"
    # Iki siniftan olustugu icin LabelEncoding yapilir.
    df_installments_payments = apply_label_encoding(df_installments_payments, ["NEW_INS_IS_LATE"])

    df_installments_payments.drop(columns=['NEW_DAY_BEFORE_END_DATE'], inplace=True)

    df_installments_payments, ip_cat = apply_one_hot_encoding(df_installments_payments)

    ip_aggregations = {
        'NUM_INSTALMENT_VERSION': ['max'],
        'NUM_INSTALMENT_NUMBER': ['max'],
        'AMT_INSTALMENT': ['sum'],
        'AMT_PAYMENT': ['sum'],
        'NEW_INSTALMENT_PAYMENT_RATE': ['min', 'max', 'mean'],
        'NEW_INS_IS_LATE': ['mean', 'sum']
    }

    for col in ip_cat:
        ip_aggregations[col] = ['mean']

    df_ip_agg = df_installments_payments.groupby(['SK_ID_CURR']).agg(ip_aggregations)

    df_ip_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in df_ip_agg.columns.tolist()])

    return df_ip_agg


def prepare_pos_cash_balance():
    conn = pymysql.connect(host='35.228.28.142', port=int(63306), user='group2', passwd='123654', db='home_credit')
    df_pos_cash_balance = pd.read_sql_query("SELECT * FROM POS_CASH_balance", conn)

    df_pos_cash_balance, pcb_cat = apply_one_hot_encoding(df_pos_cash_balance)

    pcb_aggregations = {
        'SK_ID_PREV': ['min', 'max', 'mean', 'count'],
        'MONTHS_BALANCE': ['min', 'max'],
        'CNT_INSTALMENT': ['min', 'max', 'mean'],
        'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }

    for col in pcb_cat:
        pcb_aggregations[col] = ['mean']

    df_pcb_agg = df_pos_cash_balance.groupby(['SK_ID_CURR']).agg(pcb_aggregations)
    df_pcb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in df_pcb_agg.columns.tolist()])

    return df_pcb_agg


def installment_payment_main():
    df_ip_agg = prepare_instalment_payment()  # Intaslment Payment son hali

    df_pcb_agg = prepare_pos_cash_balance()  # Pos cash balance son hali

    df_pos_ins = df_ip_agg.join(df_pcb_agg, how='inner',
                                on=['SK_ID_CURR'])  # instalment payment ve pos cash balance birlestirilmis hali

    return df_ip_agg, df_pcb_agg, df_pos_ins


def pre_processing_and_combine():
    
    with timer("Process application train"):
        df = application_train_g()
        print("application train & test shape:", df.shape)

    values = set(df.SK_ID_CURR)
    
    with timer("previous_application"):
        prev_agg_bk = previous_application_b()
        prev_agg_bk.columns = ['BK_'+col if col != 'SK_ID_CURR'  else col for col in prev_agg_bk.columns]
        print("previous_application_bk:", prev_agg_bk.shape)  
    
    df['NEW_HAS_APPLICATION'] = prev_agg_bk.SK_ID_CURR.isin(values).astype(int)
    df['NEW_HAS_APPLICATION'].fillna(0,inplace=True)    
    df['NEW_HAS_APPLICATION'].head()
    
    with timer("previous_application"):
        prev_agg = previous_application()
        print("previous_application:", prev_agg.shape)     
 
    with timer("Bureau and Bureau Balance"):
        df_final = bureau_and_bureau_balance_features()
        print("Bureau and Bureau Balance:", df_final.shape)
        
    with timer("Installment Payments"):
        df_ip_agg, df_pcb_agg, df_pos_ins = installment_payment_main()
        print("Installment Payments", df_ip_agg.shape)

    with timer("Pos Cash Balance"):
        print("Pos Cash Balance:", df_pcb_agg.shape)

    with timer("Credit Card Balance"):
        ccb = credit_card_balance()
        print("Credit Card Balance:", ccb.shape)
        
        
  
    cbb_index = DataFrame(ccb.index.values.tolist())
    df['NEW_FLAG_CCB'] = cbb_index.isin(values).astype(int)
    df['NEW_FLAG_CCB'].fillna(0,inplace=True)
    df['NEW_FLAG_CCB'].head()
    
    with timer("All tables are combining"):
       
        df1 = df.merge(df_final, how="left", on="SK_ID_CURR")
        df2 = df1.merge(df_pos_ins, how="left", on="SK_ID_CURR")
        #df2 = df1.merge(df_ip_agg, how='left', on='SK_ID_CURR')
        #df3 = df2.merge(df_pcb_agg, how='left', on='SK_ID_CURR')
        df4 = df2.merge(ccb, how='left', on='SK_ID_CURR')
        df5 = df4.merge(prev_agg_bk, how='left', on='SK_ID_CURR')
        all_df = df5.merge(prev_agg, how='left', on='SK_ID_CURR')

        print("all_df process:", all_df.shape)
 
    
    return all_df


def modeling(all_data):
    all_data = all_data.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))

    train_df = all_data[all_data['TARGET'].notnull()]
    test_df = all_data[all_data['TARGET'].isnull()]

    folds = KFold(n_splits=10, shuffle=True, random_state=1001)

    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()

    feats = [f for f in train_df.columns if f not in ['TARGET', 'SK_ID_CURR']]

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]

        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf = LGBMClassifier(
            n_jobs=-1,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=40,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=12,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
                eval_metric='auc', verbose=100, early_stopping_rounds=200)

        # y_pred_valid
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))  # y_pred_valid

    test_df['TARGET'] = sub_preds
    test_df[['SK_ID_CURR', 'TARGET']].to_csv("atilla_muhammet.csv'", index=False)

    display_importances(feature_importance_df)

    return feature_importance_df

In [5]:
def main():
    with timer("Preprocessing Time"):
        all_data = pre_processing_and_combine()
        all_data.to_csv('export_homecredit.csv', index = False, header=True)

    #with timer("Modeling"):
    #    feat_importance = modeling(all_data)

In [6]:
gc.collect()
if __name__ == "__main__":
    with timer("Full model run"):
        main()

application_tarin_g
connection finished
Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 92.38 MB
Decreased by 67.7%
Memory usage of dataframe is 45.00 MB
Memory usage after optimization is: 14.60 MB
Decreased by 67.6%
DF SAYISI:
356250
application train & test shape: (356250, 163)
Process application train - done in 104s
Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 309.01 MB
Decreased by 34.5%
Credit Application
Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 92.38 MB
Decreased by 67.7%
Credit App bağlandı
installments_payments
Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 311.40 MB
Decreased by 62.5%
installments_payments bağlandı
previous_application_bk: (278136, 97)
previous_application - done in 930s
Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 309.01 MB
Decreased by 34.5%
previous_application: (338857, 96)
previous_application - done in 181

KeyError: 'SK_ID_CURR'

########################################################

In [ ]:
export_df = reduce_mem_usage(pd.read_csv('export_homecredit.csv'))

In [ ]:
corr_features = correlation(export_df, 0.90)
print('correlated features: ', len(set(corr_features)) )
export_df.drop(labels=corr_features, axis=1, inplace=True)

In [ ]:
## Burda Kesersek 79.415
modeling(export_df)